In [2]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
import time
import undetected_chromedriver.v2 as uc
import nltk
from bs4.element import Comment
import urllib.request
import trafilatura
import string
import re

In [12]:
driver = uc.Chrome()
driver.get("https://topdev.vn/")

In [20]:
def login():
    driver.get("https://topdev.vn/")
    wait = WebDriverWait(driver, 10)
    original_window = driver.current_window_handle
#     close_ads_button = wait.until(EC.presence_of_element_located((By.ID,"iDontCareEventTopDev")))
#     close_ads_button.click()
    time.sleep(4)

#     ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)
    try:
        sign_in_button = wait.until(EC.presence_of_element_located((By.XPATH, "//a[@id = 'btn_login']")))
        sign_in_button.click()
    except:
        sign_in_button = wait.until(EC.presence_of_element_located((By.XPATH, "//li[@class = 'non-login']")))
        sign_in_button.click()
    for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            break            
    github_button = wait.until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT,"Github"))).click()
    driver.implicitly_wait(4)
    input_username =  driver.find_element(By.NAME, "login")
    input_password = driver.find_element(By.NAME, "password")
    driver.implicitly_wait(4)
    actions = ActionChains(driver)
    actions.send_keys_to_element(input_username, "chinh02").perform()
    actions.send_keys_to_element(input_password, "67729702aB").perform()
    submit_button = driver.find_element(By.XPATH,"//input[@type='submit']")
    submit_button.click()    
    driver.switch_to.window(original_window)
    current_window = driver.current_window_handle
    wait.until(EC.number_of_windows_to_be(1))


In [16]:
def get_url():
    driver.get("https://topdev.vn/viec-lam-it?src=topdev.vn&medium=mainmenu")
    for i in range (1,100):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)       
    urls =[]
    main_page_source = driver.page_source
    soup = BeautifulSoup(main_page_source,'html.parser')
    box = soup.find_all("a",{"data-list":"search-job-sp-results"}, href = True)
    for url in box:
        urls.append(url['href'])
    return urls

In [32]:
def extract_record(url_detail):
    driver.get("https://topdev.vn{}".format(url_detail))
    time.sleep(1)
    detail_pagesource = driver.page_source
    detail_soup = BeautifulSoup(detail_pagesource,'html.parser')
    try:
        jobTitle = detail_soup.find("h3",{"class":"font-size20 mb-2 c--primary"}).text.strip()
    except AttributeError:
        jobTitle = detail_soup.find("h1",{"class":"font-size20 mb-2 c--primary"}).text.strip()
    all_p_no_class = detail_soup.select("div.fixed_scroll p")
    try:
        experience = all_p_no_class[3].text.strip()
    except IndexError:
        experience =''
    try:
        position = all_p_no_class[4].text.strip()
    except IndexError:
        position = ''
    companyName = detail_soup.find("p",{"class":"fwb font-size20"}).text.strip()
    try:
        address = detail_soup.find("li",{"class":"mb-0 pb-0"}).text.strip()
        list_address = list(address.split(','))
        cleaned_address = str(list_address[-1])
    except AttributeError:
        list_address=''
        cleaned_address=''
    all_span_no_class = detail_soup.select("div.fixed_scroll span")
    try:
        jobType = all_span_no_class[1].text.strip()
    except IndexError:
        jobType =''
    gender = ''
    amount = ''
    try:
        salary = detail_soup.find("span",{"class":"job-salary-salary c--primary"}).text.strip()
    except:
        salary=''
    details=(jobTitle, companyName,salary,amount,jobType,position,gender,experience,cleaned_address)
    return details

In [22]:
driver = uc.Chrome()
login()
urls = get_url()


In [29]:

print(len(urls))


1232


In [33]:
driver = uc.Chrome()
login()
records = []
for url in urls:
    records.append(extract_record(url))

In [6]:
driver = uc.Chrome()
driver.get("https://topdev.vn/viec-lam/bb1-java-developer-1y471-ngan-hang-tmcp-hang-hai-viet-nam-msb-2029105")
detail_pagesource = driver.page_source
detail_soup = BeautifulSoup(detail_pagesource,'html.parser')
all_span_no_class = detail_soup.select("div.fixed_scroll span")
jobType = all_span_no_class[1].text.strip()
all_p_no_class = detail_soup.select("div.fixed_scroll p")
experience = all_p_no_class[3].text.strip()
position = all_p_no_class[4].text.strip()
print(experience)
print (position)
print(jobType)

3 năm
Senior
Fulltime


In [34]:
with open ('topdev.csv','w',newline='',encoding='UTF-16') as f:
    writer = csv.writer(f)
    writer.writerow(['jobTitle','companyName','salary','amount','jobType','position','gender','experience','address'])
    writer.writerows(map(lambda x: [x], records))

In [35]:
import codecs

with open('topdev.csv', newline='') as csvfile:
    csvReader = csv.reader(codecs.open('topcv.csv', 'rU', 'utf-16'))
    for row in csvReader:
        print(', '.join(row))

jobTitle, companyName, salary, amount, jobType, position, gender, experience, address
('Full Stack Developer (Reactjs/ Nodejs) Upto 40M (Net)', 'CÔNG TY CỔ PHẦN BSTAR SOLUTIONS', 'Tới 40 triệu', '10 người', 'Toàn thời gian', 'Nhân viên', 'Không yêu cầu', '3 năm', 'Hồ Chí Minh')
('IT Helpdesk', 'PIXTA Vietnam Co., Ltd.', '350-450 USD', '1 người', 'Toàn thời gian', 'Nhân viên', 'Không yêu cầu', 'Không yêu cầu kinh nghiệm', 'Hà Nội')
('Java Developer (All Levels)', 'CÔNG TY TNHH BOSCH GLOBAL SOFTWARE TECHNOLOGIES', 'Thoả thuận', '10 người', 'Toàn thời gian', 'Nhân viên', 'Không yêu cầu', '2 năm', 'Hồ Chí Minh')
('Data Engineer (Upto 25M)', 'Công ty CP công nghệ và truyền thông SAMO', 'Tới 25 triệu', '1 người', 'Toàn thời gian', 'Nhân viên', 'Không yêu cầu', '1 năm', 'Hà Nội')
('Chuyên Viên Quản Trị Ứng Dụng Thu Nhập 15M-25M Đi Làm Ngay', 'CÔNG TY TNHH NHÀ NƯỚC MỘT THÀNH VIÊN THƯƠNG MẠI VÀ XUẤT NHẬP KHẨU VIETTEL', '25-30 triệu', '1 người', 'Toàn thời gian', 'Nhân viên', 'Nam', '2 năm', 'Hà

In [2]:
str1 ="Experience Business Analyst"
str1 = str1.lower().strip()
print(str1)

experience business analyst
